# Phase 1 — Nature Inspired Computation Project

**Dataset:** ISIC (dermoscopic skin lesion images) — clinical, high-dimensional, >20k samples.

Notebook Content: dataset import, baseline model, feature extraction, Ant Colony feature selection, and four metaheuristic hyperparameter optimization methods (PSO, GA, DE, SA)

## 1) Setup — install dependencies
install packages used for metaheuristics.

In [2]:
# Install required packages
import sys
IN_COLAB = 'google.colab' in sys.modules
try:
    import tensorflow as tf
except Exception:
    !pip install --quiet tensorflow
    import tensorflow as tf
try:
    import pyswarms
except Exception:
    !pip install --quiet pyswarms deap
    import pyswarms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print('Setup done. TensorFlow version:', tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 7.1 MB/s eta 0:00:00
Setup done. TensorFlow version: 2.19.0


## 2) Dataset: download & load

Two options: (A) **Kaggle** ISIC mirror or (B) **HuggingFace** processed dataset. If you have a Kaggle token, use Kaggle; otherwise download manually and upload to runtime. Below are example commands for Kaggle.

In [4]:
import os
import shutil

# Path where your Kaggle token is currently located
kaggle_json_src = "/content/kaggle.json"
kaggle_json_dst = os.path.expanduser("~/.kaggle/kaggle.json")

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(kaggle_json_dst), exist_ok=True)

# Copy the file to the default location
shutil.copyfile(kaggle_json_src, kaggle_json_dst)

# Fix permissions (required by Kaggle API)
os.chmod(kaggle_json_dst, 0o600)

print("✅ Kaggle token copied to:", kaggle_json_dst)


✅ Kaggle token copied to: /root/.kaggle/kaggle.json


In [5]:
import os, shutil

# your existing token
src = "/content/kaggle.json"

# both possible destinations
dst1 = os.path.expanduser("~/.kaggle/kaggle.json")
dst2 = os.path.expanduser("~/.config/kaggle/kaggle.json")

# create both directories
os.makedirs(os.path.dirname(dst1), exist_ok=True)
os.makedirs(os.path.dirname(dst2), exist_ok=True)

# copy to both (safe redundancy)
shutil.copyfile(src, dst1)
shutil.copyfile(src, dst2)

# fix permissions (required)
os.chmod(dst1, 0o600)
os.chmod(dst2, 0o600)

print("✅ Copied kaggle.json to both expected paths:")
print(" -", dst1)
print(" -", dst2)


✅ Copied kaggle.json to both expected paths:
 - /root/.kaggle/kaggle.json
 - /root/.config/kaggle/kaggle.json


In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

DATA_DIR = '/content/data'
os.makedirs(DATA_DIR, exist_ok=True)

api.dataset_download_files(
    "salviohexia/isic-2019-skin-lesion-images-for-classification",
    path=DATA_DIR,
    unzip=True
)
print("✅ Download complete — files extracted to", DATA_DIR)


Dataset URL: https://www.kaggle.com/datasets/salviohexia/isic-2019-skin-lesion-images-for-classification


## 3) Preprocessing & Data generator
We use Keras `image_dataset_from_directory` if images are arranged in class folders. Otherwise load CSV mapping image->label and build TensorFlow dataset.

In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Example: if you extracted HAM10000 into DATA_DIR/ham10000
IMAGE_SIZE = (224,224)
BATCH_SIZE = 32

# If your images are arranged in subfolders per class use this (uncomment and set path):
# train_ds = image_dataset_from_directory(os.path.join(DATA_DIR,'train'), image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, shuffle=True, seed=123)
# val_ds = image_dataset_from_directory(os.path.join(DATA_DIR,'val'), image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

print('Define data loader according to your local filesystem. If using ISIC CSV, load the CSV and build tf.data.Dataset mapping filenames to labels.')

## 4) Baseline model (transfer learning)
We build an EfficientNetB0-based classifier with a small dense head. This is our baseline before metaheuristic tuning.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

def build_baseline_model(num_classes, input_shape=(224,224,3), dropout=0.3, lr=1e-4):
    base = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base.input, outputs=out)
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

print('Baseline model builder ready (EfficientNetB0)')

## 5) Feature extraction pipeline (for ACO feature selection)
We extract deep embeddings (GlobalAveragePooling2D outputs) from the pretrained backbone, producing a fixed-length feature vector per image (e.g., 1280 dims for EfficientNetB0). Then we run Ant Colony Optimization to select a subset of those features.

In [ ]:
def extract_features(model_backbone, dataset, batch_size=32):
    """Given a model that outputs embeddings (e.g., base+gap), return X (n_samples x d) and y."""
    features = []
    labels = []
    for batch in dataset:  # dataset yields (images, labels)
        imgs, labs = batch
        feats = model_backbone.predict(imgs)
        features.append(feats)
        labels.append(labs.numpy())
    X = np.vstack(features)
    y = np.concatenate(labels)
    return X, y

print('Feature extraction helper ready')

### 5.a Build an embedding model (backbone->GAP)
We'll reuse EfficientNetB0 backbone and create a model that returns the pooled features.

In [ ]:
from tensorflow.keras.models import Model
backbone = EfficientNetB0(include_top=False, input_shape=(224,224,3), weights='imagenet')
gap = layers.GlobalAveragePooling2D()(backbone.output)
embed_model = Model(inputs=backbone.input, outputs=gap)
print('Embedding model ready — output dim =', embed_model.output_shape)

## 6) Ant Colony Optimization (ACO) for feature selection
We apply ACO to select a subset of embedding features that maximize validation accuracy when training a small classifier on those features. For clarity we implement a simple ACO variant tailored to feature selection.

In [ ]:
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def aco_feature_selection(X, y, n_ants=20, n_iters=25, alpha=1.0, beta=2.0, rho=0.1, k_features=50):
    """Simple ACO for selecting k_features indices from feature vector X. Returns best_indices.
    - X: (n_samples, n_features)
    - uses logistic regression as quick evaluator (CV accuracy)
    """
    n_features = X.shape[1]
    # initialize pheromone
    pheromone = np.ones(n_features)
    best_solution = None
    best_score = 0.0
    for it in range(n_iters):
        all_solutions = []
        all_scores = []
        for ant in range(n_ants):
            probs = (pheromone ** alpha)
            probs = probs / probs.sum()
            # stochastic selection proportionally to pheromone
            chosen = np.random.choice(np.arange(n_features), size=k_features, replace=False, p=probs)
            Xsub = X[:, chosen]
            clf = LogisticRegression(max_iter=500)
            # quick 3-fold CV to score
            try:
                score = cross_val_score(clf, Xsub, y, cv=3, scoring='accuracy').mean()
            except Exception:
                score = 0.0
            all_solutions.append(chosen)
            all_scores.append(score)
            if score > best_score:
                best_score = score
                best_solution = chosen
        # pheromone evaporation
        pheromone = (1 - rho) * pheromone
        # deposit pheromone according to quality
        for sol, sc in zip(all_solutions, all_scores):
            for idx in sol:
                pheromone[idx] += sc
        print(f'Iter {it+1}/{n_iters} best_score={best_score:.4f}')
    return np.array(best_solution), best_score

print('ACO feature selection function ready')

## 7) Metaheuristic hyperparameter optimization wrappers
We will provide wrappers to tune a small set of hyperparameters of the classifier model: e.g., learning_rate, dropout, number_of_dense_units (or top layers), and number of fine-tuned layers. For Phase 1 we run four algorithms: PSO, Genetic Algorithm (GA using DEAP), Differential Evolution (scipy), and Simulated Annealing (custom).

In [ ]:
# Example objective: given a vector x (hyperparameters), build model, train for few epochs on small subset, return validation loss or negative accuracy.
def decode_vector(x):
    """Decode float vector to meaningful hyperparameters.
    x: array [lr_log, dropout, dense_units_norm, fine_tune_frac]
    returns: dict
    """
    lr = 10 ** (-5 + x[0] * 3)      # maps [0,1] -> [1e-5,1e-2]
    dropout = 0.05 + x[1] * 0.5    # maps [0,1] -> [0.05,0.55]
    dense = int(64 + x[2] * 448)   # [64,512]
    ft_frac = x[3]
    return {'lr': lr, 'dropout': dropout, 'dense': dense, 'ft_frac': ft_frac}

def objective_fn(x, train_ds, val_ds, num_classes):
    params = decode_vector(x)
    # Build model with params
    model = build_baseline_model(num_classes, dropout=params['dropout'], lr=params['lr'])
    # Optionally unfreeze top layers according to ft_frac
    total_layers = len(model.layers)
    n_unfreeze = int(total_layers * params['ft_frac'])
    if n_unfreeze > 0:
        for layer in model.layers[-n_unfreeze:]:
            try:
                layer.trainable = True
            except Exception:
                pass
    # Train for a few epochs (fast eval)
    history = model.fit(train_ds, validation_data=val_ds, epochs=3, verbose=0)
    val_acc = history.history['val_accuracy'][-1]
    # Return negative val_acc as minimizer
    return -float(val_acc)

print('Objective and decoder ready')

### 7.a PSO using pyswarms
We search in [0,1]^4 space which the decode_vector maps into hyperparameters.

In [ ]:
import pyswarms as ps
def run_pso(train_ds, val_ds, num_classes, n_particles=12, iters=20):
    def f(x):
        # pyswarms passes x as (n_particles, dimensions)
        out = []
        for row in x:
            out.append(objective_fn(row, train_ds, val_ds, num_classes))
        return np.array(out)
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
    optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=4, options=options)
    best_cost, best_pos = optimizer.optimize(f, iters=iters)
    return best_pos, best_cost

print('PSO wrapper ready')

### 7.b Differential Evolution (scipy)
Use `scipy.optimize.differential_evolution` on the 4-d box [0,1]^4. It expects a function that returns scalar to minimize.

In [ ]:
from scipy.optimize import differential_evolution
def run_de(train_ds, val_ds, num_classes, maxiter=10):
    bounds = [(0,1)]*4
    def f(x):
        return objective_fn(x, train_ds, val_ds, num_classes)
    res = differential_evolution(f, bounds, maxiter=maxiter, polish=False)
    return res.x, res.fun

print('DE wrapper ready')

### 7.c Genetic Algorithm (DEAP)
We use DEAP to implement a simple GA searching in [0,1]^4.

In [ ]:
import random
from deap import base, creator, tools, algorithms

def run_ga(train_ds, val_ds, num_classes, ngen=10, pop=12):
    creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
    creator.create('Individual', list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register('attr_float', random.random)
    toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_float, n=4)
    toolbox.register('population', tools.initRepeat, list, toolbox.individual)
    def eval_ind(ind):
        return (objective_fn(np.array(ind), train_ds, val_ds, num_classes),)
    toolbox.register('evaluate', eval_ind)
    toolbox.register('mate', tools.cxBlend, alpha=0.5)
    toolbox.register('mutate', tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
    toolbox.register('select', tools.selTournament, tournsize=3)
    popu = toolbox.population(n=pop)
    hof = tools.HallOfFame(1)
    algorithms.eaSimple(popu, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, halloffame=hof, verbose=True)
    best = hof.items[0]
    return np.array(best), best.fitness.values[0]

print('GA wrapper ready (DEAP)')

### 7.d Simulated Annealing (custom)
Simple annealing on the 4-d continuous space with gaussian proposals.


In [ ]:
def run_sa(train_ds, val_ds, num_classes, n_iters=50, T0=1.0, alpha=0.95):
    # start from random
    x = np.random.rand(4)
    fx = objective_fn(x, train_ds, val_ds, num_classes)
    best_x, best_fx = x.copy(), fx
    T = T0
    for i in range(n_iters):
        cand = x + np.random.normal(scale=0.1, size=4)
        cand = np.clip(cand, 0, 1)
        f_cand = objective_fn(cand, train_ds, val_ds, num_classes)
        if f_cand < fx or np.random.rand() < np.exp((fx - f_cand)/T):
            x, fx = cand, f_cand
            if fx < best_fx:
                best_x, best_fx = x.copy(), fx
        T *= alpha
    return best_x, best_fx

print('SA wrapper ready')

## 8) Evaluation & comparison
After running each optimization, record: best hyperparameters, validation accuracy, training time (per trial), and optionally confusion matrix on test set.

## 9) Notes on reproducibility and runtime
 - The notebook focuses on a reproducible pipeline. For hyperparameter search we train for a few epochs for fast evaluation; for the final selected hyperparameters, retrain with more epochs and possibly unfreeze backbone.
 - If dataset is large, consider sampling a subset for fast metaheuristic runs, or use small image size during search.
 - Placeholders for visualization (training curves, confusion matrix, Grad-CAM) are included in the report/presentation code below.